<a href="https://colab.research.google.com/github/davidclizbe/datascience/blob/master/ClizbeInterprettingCoefficientspynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Interpretation and significance
Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:

 

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873+0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒+0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒2−223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠expenditure=873+0.0012annual_income+0.00002annual_income2−223.57have_kids

expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars, and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer.


Descriptive statistics, including a correlation matrix. and likely a root mean square error to assess model error/performance. 

2. Weather model
In this exercise, you'll work with the historical temperature data from the previous checkpoint. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

First, load the dataset from the weatherinszeged table from Thinkful's database.
Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables?
Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weatherinszeged = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
weatherinszeged

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.
...,...,...,...,...,...,...,...,...,...,...,...,...
96448,2016-09-09 17:00:00+00:00,Partly Cloudy,rain,26.016667,26.016667,0.43,10.9963,31.0,16.1000,0.0,1014.36,Partly cloudy starting in the morning.
96449,2016-09-09 18:00:00+00:00,Partly Cloudy,rain,24.583333,24.583333,0.48,10.0947,20.0,15.5526,0.0,1015.16,Partly cloudy starting in the morning.
96450,2016-09-09 19:00:00+00:00,Partly Cloudy,rain,22.038889,22.038889,0.56,8.9838,30.0,16.1000,0.0,1015.66,Partly cloudy starting in the morning.
96451,2016-09-09 20:00:00+00:00,Partly Cloudy,rain,21.522222,21.522222,0.60,10.5294,20.0,16.1000,0.0,1015.95,Partly cloudy starting in the morning.


In [ ]:
weatherinszeged['tempdiff'] = weatherinszeged['apparenttemperature'] - weatherinszeged['temperature']
weatherinszeged.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary,tempdiff
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.,-2.083333
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.,-2.127778
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.,0.000000
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.,-2.344444
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.,-1.777778


In [ ]:
weatherdata = pd.get_dummies(weatherinszeged, dummy_na=True)
weatherdata.head()

,date,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,tempdiff,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,dailysummary_nan
0,2006-03-31 22:00:00+00:00,

In [ ]:
#weatherdata.dropna(inplace=True)

In [ ]:
weatherdata = weatherdata.replace([np.inf, -np.inf], np.nan).dropna()

In [ ]:
y = weatherdata['tempdiff']

X = weatherdata.drop(['tempdiff','temperature', 'apparenttemperature'], axis=1)
X.head()

,date,humidity,windspeed,windbearing,visibility,loudcover,pressure,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,preciptype_snow,preciptype_nan,dailysummary_Breezy and foggy starting in the evening.,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,

In [ ]:
y.head()

0   -2.083333
1   -2.127778
2    0.000000
3   -2.344444
4   -1.777778
Name: tempdiff, dtype: float64

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,date,humidity,windspeed,windbearing,visibility,loudcover,pressure,summary_Breezy,summary_Breezy and Dry,summary_Breezy and Foggy,summary_Breezy and Mostly Cloudy,summary_Breezy and Overcast,summary_Breezy and Partly Cloudy,summary_Clear,summary_Dangerously Windy and Partly Cloudy,summary_Drizzle,summary_Dry,summary_Dry and Mostly Cloudy,summary_Dry and Partly Cloudy,summary_Foggy,summary_Humid and Mostly Cloudy,summary_Humid and Overcast,summary_Humid and Partly Cloudy,summary_Light Rain,summary_Mostly Cloudy,summary_Overcast,summary_Partly Cloudy,summary_Rain,summary_Windy,summary_Windy and Dry,summary_Windy and Foggy,summary_Windy and Mostly Cloudy,summary_Windy and Overcast,summary_Windy and Partly Cloudy,summary_nan,preciptype_null,preciptype_rain,preciptype_snow,preciptype_nan,dailysummary_Breezy and foggy starting in the evening.,...,dailysummary_Partly cloudy starting overnight continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the evening.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening and windy starting in the morning continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until evening.,dailysummary_Partly cloudy starting overnight continuing until morning and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until morning.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the afternoon.,dailysummary_Partly cloudy starting overnight continuing until night and breezy in the morning.,dailysummary_Partly cloudy starting overnight continuing until night and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy starting overnight continuing until night.,dailysummary_Partly cloudy starting overnight.,dailysummary_Partly cloudy throughout the day and breezy in the afternoon.,dailysummary_Partly cloudy throughout the day and breezy in the evening.,dailysummary_Partly cloudy throughout the day and breezy in the morning.,dailysummary_Partly cloudy throughout the day and breezy starting in the afternoon continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy throughout the day and breezy starting in the morning continuing until night.,dailysummary_Partly cloudy throughout the day and windy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy throughout the day.,dailysummary_Partly cloudy until afternoon.,dailysummary_Partly cloudy until evening and breezy in the afternoon.,dailysummary_Partly cloudy until evening and breezy in the morning.,dailysummary_Partly cloudy until evening and breezy overnight.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until evening and breezy starting in the morning continuing until evening.,dailysummary_Partly cloudy until evening.,dailysummary_Partly cloudy until morning.,dailysummary_Partly cloudy until night and breezy in the afternoon.,dailysummary_Partly cloudy until night and breezy in the morning.,dailysummary_Partly cloudy until night and breezy starting in the morning continuing until afternoon.,dailysummary_Partly cloudy until night.,dailysummary_Rain throughout the day.,dailysummary_Rain until afternoon.,dailysummary_Rain until morning.,dailysummary_Windy in the afternoon.,

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77162 entries, 33716 to 42486
Columns: 254 entries, date to dailysummary_nan
dtypes: datetime64[ns, UTC](1), float64(6), uint8(247)
memory usage: 22.9 MB


In [ ]:
import statsmodels.api as sm
import numpy as np
X_train = np.array(X_train)
X_train = sm.add_constant(X_train)
y_train = np.array(y_train)
model = sm.OLS(y_train, X_train)
results = model.fit()
results.params

TypeError: ignored

In [ ]:
results.tvalues

3. House prices model
In this exercise, you'll interpret your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

Load the houseprices data from Thinkful's database.
Run your house prices model again and interpret the results. Which features are statistically significant, and which are not?
Now, exclude the insignificant features from your model. Did anything change?
Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have a more prominent effect on house prices?
Do the results sound reasonable to you? If not, try to explain the potential reasons.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

houseprices = pd.read_sql_query('select * from houseprices',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
houseprices

In [ ]:
houseprices.corr()

In [ ]:
housedata = pd.get_dummies(houseprices, dummy_na=True)
housedata.head()

In [ ]:
housedata.dropna(inplace=True)

In [ ]:
y = housedata['saleprice']

X = housedata.drop(['saleprice', 'overallqual', 'yearbuilt'], axis=1)
X.head()

In [ ]:
y.head()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

In [ ]:
import statsmodels.api as sm
import numpy as np

model = sm.OLS(y_train,X_train)
results = model.fit()
results.params

In [ ]:
results.tvalues

In [ ]:
X_test.shape

In [ ]:
y_test.shape

In [ ]:
y_pred = results.predict(X_test)

In [ ]:
results.score(y_test, y_pred)